# This is taking a lot longer than I remember

I'm pretty sure using a pretrained model is the way to go.  There aren't enough images to create a model from scratch

Some things to try:
- Make more layers untrainable
- See if I can look at the geometry of the pretrained model
- Try different pretrained models like VGG19
- Flatten the input data to something the model likes more

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
from keras.applications import ResNet50
from keras.models import Model

num_categories = 5
image_w = 320
image_h = 240

# This is how the Keras documentation suggests performing transfer learning
base_model = ResNet50(include_top = False,
               pooling = 'avg', 
               input_shape = (image_w, image_h, 3))

# add a global spatial average pooling layer
x = base_model.output

# let's add a fully-connected layer
# x = Dense(256, activation='relu')(x)

predictions = Dense(num_categories, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

model.layers[0].trainable = False
model.layers[1].trainable = False
model.layers[2].trainable = False
model.layers[3].trainable = False

model.compile(optimizer = "adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

Using TensorFlow backend.


In [3]:
len(model.layers)

176

In [4]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                             horizontal_flip = True,
                                             vertical_flip = True,
                                             rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
                                             zoom_range = 0.1, # Randomly zoom image 
                                             width_shift_range = 0.2,
                                             height_shift_range = 0.2)


train_generator = data_generator_with_aug.flow_from_directory(
        '../data/flowers/train/',
        target_size = (image_w, image_h),
        batch_size = 20,
        class_mode='categorical')

validation_generator = data_generator_with_aug.flow_from_directory(
        '../data/flowers/val/',
        target_size = (image_w, image_h),
        class_mode = 'categorical')

Found 3244 images belonging to 5 classes.
Found 1079 images belonging to 5 classes.


In [5]:
history = model.fit_generator(
        train_generator,
        epochs = 5,
        validation_data=validation_generator)

Epoch 1/5
 14/649 [..............................] - ETA: 1:51:56 - loss: 2.6437 - acc: 0.3714

KeyboardInterrupt: 